# Zad 1

Dane są tabele: 

> • Towary(ID, NazwaTowaru), 

> • Ceny(TowarID REF Towary(ID), Waluta REF Kursy(Waluta), Cena), 

> • Kursy(Waluta, CenaPLN). 

Należy zwrócić uwagę, że towar może nie mieć podanej ceny we wszystkich walutach, ale zawsze ma podaną cenę w PLN (jest to punkt odniesienia). Zadaniem jest przygotowanie wsadu, który zaktualizuje ceny w tabeli Ceny na podstawie tabeli Kursy, przy czym może się zdarzyć, że w tabeli Ceny będzie odniesienie do waluty, której kursu w tabeli Kursy już nie ma, i o taką sytuację również należy zadbać. 

Przy realizacji tego zadania należy wykorzystać mechanizm kursorów.

In [1]:
-- create tables
DROP TABLE IF EXISTS Ceny, Kursy, Towary
GO

CREATE TABLE Towary(
    ID INT PRIMARY KEY, 
    NazwaTowaru VARCHAR(40)
)
GO

CREATE TABLE Kursy(
    Waluta VARCHAR(3) PRIMARY KEY, 
    CenaPLN MONEY
)
GO

-- zamiana kolejności bo ceny zależą od kursów
CREATE TABLE Ceny(
    TowarID INT REFERENCES Towary(ID),
    Waluta VARCHAR(3) REFERENCES Kursy(Waluta), 
    Cena MONEY
)
GO

-- insert dummy data
INSERT INTO Towary
VALUES
    (1, 'chleb 1000g'),
    (2, 'bułka kajzerka'),
    (3, 'mleko 1L'),
    (4, 'ser żółty gouda 1 kg'),
    (5, 'jogurt 150g')

INSERT INTO Kursy
VALUES
    ('PLN', 1.0),
    ('USD', 4.3125),
    ('GBP', 5.6769),
    ('EUR', 4.7459),
    ('RUB', 0.0432)

INSERT INTO Ceny
VALUES
    (1, 'PLN', 4.20),
    (2, 'PLN', 0.33),
    (3, 'PLN', 3.16),
    (4, 'PLN', 21.36),
    (5, 'PLN', 1.36),
    (1, 'GBP', 0.7398),
    (3, 'USD', 0.55664),
    (4, 'RUB', 494.44),
    (5, 'GBP', 20.2395)
GO

SELECT * FROM Towary;
SELECT * FROM Ceny;
SELECT * FROM Kursy;

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(5 rows affected)

...

(5 rows affected)

Total execution time: 00:00:00.460

ID,NazwaTowaru
1,chleb 1000g
2,bulka kajzerka
3,mleko 1L
4,ser zólty gouda 1 kg
5,jogurt 150g


TowarID,Waluta,Cena
1,PLN,"4,20"
2,PLN,"0,33"
3,PLN,"3,16"
4,PLN,"21,36"
5,PLN,"1,36"
1,GBP,"0,7398"
3,USD,"0,5566"
4,RUB,"494,44"
5,GBP,"20,2395"


Waluta,CenaPLN
EUR,"4,7459"
GBP,"5,6769"
PLN,"1,00"
RUB,"0,0432"
USD,"4,3125"


In [80]:
ALTER TABLE Ceny NOCHECK CONSTRAINT ALL

DELETE FROM Kursy
WHERE
    Waluta = 'RUB'

ALTER TABLE Ceny CHECK CONSTRAINT ALL

SELECT * FROM Kursy

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.080

Waluta,CenaPLN
EUR,"4,7459"
GBP,"5,6769"
PLN,"1,00"
USD,"4,3125"


In [81]:
-- kursory
DECLARE Ceny_kursor CURSOR FOR SELECT * FROM dbo.Ceny
DECLARE Kursy_kursor CURSOR FOR SELECT Waluta, CenaPLN FROM dbo.Kursy
GO

-- zmienne dla kursorów
DECLARE @Kursy_Waluta VARCHAR(3), @Kursy_CenaPLN MONEY
DECLARE @Ceny_TowarID INT, @Ceny_Waluta VARCHAR(3), @Ceny_Cena MONEY

DECLARE @cena MONEY; -- cena dla obecnego produktru
DECLARE @to_delete BIT; -- usuwamy gdy brakuje przelicznika kursu dla obecnego produktu i kursu

-- przechodzimy po towarach na liście towar-waluta-cena
OPEN Ceny_kursor
FETCH NEXT FROM Ceny_kursor INTO @Ceny_TowarID, @Ceny_Waluta, @Ceny_Cena
WHILE (@@FETCH_STATUS = 0) -- są jeszcze towary na liście (towar - cena)
BEGIN
    -- złoty jest bazową walutą
    IF (@Ceny_Waluta = 'PLN')
    BEGIN
        FETCH NEXT FROM Ceny_kursor INTO @Ceny_TowarID, @Ceny_Waluta, @Ceny_Cena
        CONTINUE
    END
    
    -- bazowa cena (w PLN)
    SET @cena = (SELECT TOP 1 Cena 
                 FROM dbo.Ceny 
                 WHERE TowarID = @Ceny_TowarID AND Waluta = 'PLN'
    )
    
    -- forced cursor usage
    -- przejrzyj czy istnieje taki kurs jak tak to update, jak nie to delete
    SET @to_delete = 1

    OPEN Kursy_kursor    
    FETCH NEXT FROM Kursy_kursor INTO @Kursy_Waluta, @Kursy_CenaPLN
    WHILE (@@FETCH_STATUS = 0) 
    BEGIN
        IF (@Kursy_Waluta != @Ceny_Waluta)
        BEGIN
            FETCH NEXT FROM  Kursy_kursor INTO @Kursy_Waluta, @Kursy_CenaPLN
            CONTINUE
        END
        
        SET @Ceny_Cena = @cena / @Kursy_CenaPLN
        SET @to_delete = 0

        UPDATE Ceny 
        SET Cena = @Ceny_Cena
        WHERE TowarID = @Ceny_TowarID AND Waluta = @Ceny_Waluta

        FETCH NEXT FROM  Kursy_kursor INTO @Kursy_Waluta, @Kursy_CenaPLN
    END

    IF (@to_delete = 1)
        DELETE FROM Ceny WHERE TowarID = @Ceny_TowarID AND Waluta = @Ceny_Waluta
    
    CLOSE Kursy_kursor
    FETCH NEXT FROM Ceny_kursor INTO @Ceny_TowarID, @Ceny_Waluta, @Ceny_Cena
END
-- pamiętaj o zamykaniu i dealokacji kursorów
CLOSE Ceny_kursor
GO

DEALLOCATE Ceny_kursor
DEALLOCATE Kursy_kursor

Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Commands completed successfully.

Total execution time: 00:00:00.213

In [84]:
-- przykład
-- rubel zostaje usunięty bo nie ma dla niego kursu (ktoś nie nadąża zmniejszać :) )
-- reszta cen w innych walutach niż PLN zostaje zaktualizowana na podstawie kursu
SELECT * FROM Towary;
SELECT * FROM Ceny;
SELECT * FROM Kursy;

(5 rows affected)

(8 rows affected)

(4 rows affected)

Total execution time: 00:00:00.053

ID,NazwaTowaru
1,chleb 1000g
2,bulka kajzerka
3,mleko 1L
4,ser zólty gouda 1 kg
5,jogurt 150g


TowarID,Waluta,Cena
1,PLN,"4,20"
2,PLN,"0,33"
3,PLN,"3,16"
4,PLN,"21,36"
5,PLN,"1,36"
1,GBP,"0,7398"
3,USD,"0,7327"
5,GBP,"0,2395"


Waluta,CenaPLN
EUR,"4,7459"
GBP,"5,6769"
PLN,"1,00"
USD,"4,3125"


In [26]:
DROP TABLE IF EXISTS Ceny, Kursy, Towary

Commands completed successfully.

Total execution time: 00:00:00.079